# Models built for data prediction

In [57]:
from _Setup import *

In [58]:
# Import Data
train = pd.read_csv(missForestTrainFile)
test = pd.read_csv(missForestTestFile)

In [59]:
X_train = train.drop(["SepsisLabel", "Unit2", "HospAdmTime", "Unnamed: 0"], axis = 1)
y_train = train['SepsisLabel']

In [60]:
train.head()

,Unnamed: 0,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,...,Shock,patient,Gender,Unit1,Unit2,Fever,TCP,TCA,LEU,SepsisLabel
0,0,0.273345,-0.602957,-0.046072,-0.497930,-1.547563,-0.313713,0.311247,0.057956,0.046142,...,-0.002114,26093,1.0,1.0,0.0,0,0,0,0,0.0
1,1,0.465811,0.538678,-0.046072,0.861707,-0.238934,-0.283314,-0.273557,0.441245,-2.034285,...,-0.002114,12185,1.0,0.0,1.0,0,1,1,1,0.0
2,2,0.562044,0.196187,0.587518,-0.633894,0.790806,1.297436,1.042253,0.632889,-1.268027,...,-0.002114,15726,0.0,1.0,0.0,0,1,0,0,0.0
3,3,-0.143664,-0.203385,-1.313251,-0.348370,-0.710899,-0.161718,0.018845,1.399466,-0.642903,...,-0.002114,25794,0.0,1.0,0.0,0,1,0,1,0.0
4,4,-0.528596,-1.630429,-1.313251,-0.797050,-0.410558,-1.316881,-1.297349,-0.516977,0.400633,...,-0.002114,22866,1.0,0.0,1.0,0,0,0,0,0.0


In [61]:
test.head()

,Unnamed: 0,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,...,ICULOS,Shock,patient,Gender,Unit1,Unit2,Fever,TCP,TCA,LEU
0,0,-0.892631,1.261514,0.243716,0.881155,0.026305,-0.256789,-0.354120,-0.545934,-0.006012,...,-0.889804,-0.002287,6560,1.0,1.0,0.0,0,0,1,0
1,1,-0.873375,0.882955,-0.369389,0.881155,-0.938603,-0.767075,-0.604890,-0.545934,-0.012025,...,-0.870382,-0.002287,6560,1.0,1.0,0.0,0,0,1,0
2,2,-0.854119,0.991115,-0.369389,0.881155,0.068257,-0.399522,-0.481548,-0.545934,-0.012025,...,-0.850959,-0.002287,6560,1.0,1.0,0.0,0,0,1,0
3,3,-0.834862,1.694153,-0.982495,0.881155,1.117070,0.764394,0.502199,-0.545934,-0.012025,...,-0.831537,-0.002287,6560,1.0,1.0,0.0,0,0,1,0
4,4,-0.815606,0.666635,-0.982495,0.674267,-1.064460,-1.114412,-0.905199,-0.545934,-0.012025,...,-0.812114,-0.002287,6560,1.0,1.0,0.0,0,0,1,0


In [62]:
test = test.drop(["Unit2", "HospAdmTime", "Unnamed: 0"], axis = 1)


In [63]:
test.head()

,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,...,Age,ICULOS,Shock,patient,Gender,Unit1,Fever,TCP,TCA,LEU
0,-0.892631,1.261514,0.243716,0.881155,0.026305,-0.256789,-0.354120,-0.545934,-0.006012,0.034508,...,1.181226,-0.889804,-0.002287,6560,1.0,1.0,0,0,1,0
1,-0.873375,0.882955,-0.369389,0.881155,-0.938603,-0.767075,-0.604890,-0.545934,-0.012025,0.029438,...,1.181226,-0.870382,-0.002287,6560,1.0,1.0,0,0,1,0
2,-0.854119,0.991115,-0.369389,0.881155,0.068257,-0.399522,-0.481548,-0.545934,-0.012025,0.023495,...,1.181226,-0.850959,-0.002287,6560,1.0,1.0,0,0,1,0
3,-0.834862,1.694153,-0.982495,0.881155,1.117070,0.764394,0.502199,-0.545934,-0.012025,0.017174,...,1.181226,-0.831537,-0.002287,6560,1.0,1.0,0,0,1,0
4,-0.815606,0.666635,-0.982495,0.674267,-1.064460,-1.114412,-0.905199,-0.545934,-0.012025,0.012087,...,1.181226,-0.812114,-0.002287,6560,1.0,1.0,0,0,1,0


In [64]:
X_train.head()

,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,...,Age,ICULOS,Shock,patient,Gender,Unit1,Fever,TCP,TCA,LEU
0,0.273345,-0.602957,-0.046072,-0.497930,-1.547563,-0.313713,0.311247,0.057956,0.046142,-0.003627,...,1.033013,0.258739,-0.002114,26093,1.0,1.0,0,0,0,0
1,0.465811,0.538678,-0.046072,0.861707,-0.238934,-0.283314,-0.273557,0.441245,-2.034285,-0.000452,...,-0.122220,0.644685,-0.002114,12185,1.0,0.0,0,1,1,1
2,0.562044,0.196187,0.587518,-0.633894,0.790806,1.297436,1.042253,0.632889,-1.268027,-0.038597,...,-2.666159,0.548199,-0.002114,15726,0.0,1.0,0,1,0,0
3,-0.143664,-0.203385,-1.313251,-0.348370,-0.710899,-0.161718,0.018845,1.399466,-0.642903,0.025856,...,0.590931,-0.159370,-0.002114,25794,0.0,1.0,0,1,0,1
4,-0.528596,-1.630429,-1.313251,-0.797050,-0.410558,-1.316881,-1.297349,-0.516977,0.400633,-0.000452,...,1.163393,-0.513154,-0.002114,22866,1.0,0.0,0,0,0,0


In [65]:
# Transform the data using the first two principal components
# pca_2d = PCA(n_components=2)
# pca_2d_data = pca_2d.fit_transform(X_train)
# 
# # Plot the first two principal components
# plt.figure(figsize=(10, 6))
# plt.scatter(pca_2d_data[:, 0], pca_2d_data[:, 1], alpha=0.5)
# plt.title('PCA - First Two Principal Components')
# plt.xlabel('Principal Component 1')
# plt.ylabel('Principal Component 2')
# plt.grid()
# plt.show()



In [66]:
# from linearmodels.panel import PanelOLS
# import statsmodels.api as sm
# 
# # Set the multi-index for your panel data
# df = resampled_train.set_index(['patient', 'time'])
# 
# # Prepare the dependent variable (y) and independent variables (X)
# X = df[['Glucose', 'Fibrinogen', 'Bilirubin_total']]  # Remove 'Age' if it does not vary within entities
# X = sm.add_constant(X)  # Add a constant term
# 
# y = df['SepsisLabel']  # Your binary dependent variable
# 
# # Combine y and X into a single DataFrame for the formula
# df_combined = X.copy()
# df_combined['SepsisLabel'] = y
# 
# # Fit the model using the fixed effects estimator
# model = PanelOLS.from_formula('SepsisLabel ~ Glucose + Fibrinogen + Bilirubin_total + EntityEffects', data=df_combined)
# 
# # Get the results without drop_absorbed
# results = model.fit()
# 
# # Display the results
# print(results)

In [67]:
# TODO : Separate train and test by patients here
X_ydf = train.drop(["Unit2", "HospAdmTime", "Unnamed: 0"], axis = 1)
X_ydf['SepsisLabel'] = X_ydf['SepsisLabel'].astype(int)

random.seed(8)

patients = X_ydf['patient'].unique()

testPatients = np.random.choice(X_ydf['patient'].unique(), size = int(patients.shape[0]/5), replace=False)

In [68]:
X_train_ydf = X_ydf[~X_ydf['patient'].isin(testPatients)]
X_test_ydf = X_ydf[X_ydf['patient'].isin(testPatients)]

In [69]:
X_test_ydf.shape

(226462, 33)

In [70]:
X_train_ydf.head()

,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,...,ICULOS,Shock,patient,Gender,Unit1,Fever,TCP,TCA,LEU,SepsisLabel
0,0.273345,-0.602957,-0.046072,-0.497930,-1.547563,-0.313713,0.311247,0.057956,0.046142,-0.003627,...,0.258739,-0.002114,26093,1.0,1.00000,0,0,0,0,0
5,0.818665,-0.774203,0.904313,-1.096171,0.104312,-0.283314,-1.223865,-0.708621,-0.990749,-0.013605,...,0.805496,-0.002114,24161,1.0,0.00004,0,0,0,0,0
6,-0.560673,-0.831284,-0.362866,-1.449676,-0.582181,-0.708901,-0.273557,-0.516977,0.058138,-0.003627,...,-0.577479,-0.002114,14053,1.0,0.00004,1,0,0,0,0
7,0.529966,-1.059611,0.587518,-0.361967,0.447559,0.932648,1.480857,-0.133688,0.048926,-0.003627,...,0.516036,-0.002114,23690,1.0,0.00000,0,0,0,0,0
9,-0.721062,-0.831284,0.904313,0.562587,0.876618,-0.648102,-1.223865,-0.900265,-1.338594,-0.000452,...,-0.738290,-0.002114,22801,1.0,0.00000,0,0,0,0,0


In [71]:
ydf_model = ydf.GradientBoostedTreesLearner(label='SepsisLabel').train(X_train_ydf, verbose = 2)

Train model on 892113 examples
Model trained in 0:03:06.090698


In [72]:
ydf_model.describe()

In [86]:
X_train_ydf.head()

,time,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,...,ICULOS,Shock,patient,Gender,Unit1,Fever,TCP,TCA,LEU,SepsisLabel
0,0.273345,-0.602957,-0.046072,-0.497930,-1.547563,-0.313713,0.311247,0.057956,0.046142,-0.003627,...,0.258739,-0.002114,26093,1.0,1.00000,0,0,0,0,0
5,0.818665,-0.774203,0.904313,-1.096171,0.104312,-0.283314,-1.223865,-0.708621,-0.990749,-0.013605,...,0.805496,-0.002114,24161,1.0,0.00004,0,0,0,0,0
6,-0.560673,-0.831284,-0.362866,-1.449676,-0.582181,-0.708901,-0.273557,-0.516977,0.058138,-0.003627,...,-0.577479,-0.002114,14053,1.0,0.00004,1,0,0,0,0
7,0.529966,-1.059611,0.587518,-0.361967,0.447559,0.932648,1.480857,-0.133688,0.048926,-0.003627,...,0.516036,-0.002114,23690,1.0,0.00000,0,0,0,0,0
9,-0.721062,-0.831284,0.904313,0.562587,0.876618,-0.648102,-1.223865,-0.900265,-1.338594,-0.000452,...,-0.738290,-0.002114,22801,1.0,0.00000,0,0,0,0,0


In [87]:
model_evaluated = ydf_model.evaluate(X_test_ydf)

In [88]:
model_evaluated

Label \ Pred,0,1
0,214926,9169
1,1669,698


In [90]:
mat = model_evaluated.confusion_matrix
vals = mat.matrix

precision = vals[1][1] / (vals[1][1] + vals[1][0])
recall = vals[1][1] / (vals[1][1] + vals[0][1])

f1 = (2*precision*recall)/(precision+recall)
print(precision, recall, f1)

0.07074085334954901 0.2948880439374736 0.11410822298512344


In [77]:
yHatTrain = ydf_model.predict(X_train_ydf)

In [78]:
# Tuning the threshold for classification
from sklearn.metrics import f1_score

for i in np.arange(0.2, 0.8, 0.05):
    yHatTrainScaled = np.where(yHatTrain >= i, 1, 0)
    print(i, " ", f1_score(y_train, yHatTrainScaled))

ValueError: Found input variables with inconsistent numbers of samples: [1118575, 892113]

In [79]:
# Predict labels
yHatTest = ydf_model.predict(test)

In [80]:
yHatTestScaled = np.where(yHatTest >= 0.5, 1, 0)

In [81]:
test_submission = pd.read_csv(rawSubmissionFile)
test_submission.head()

,ID,SepsisLabel
0,6560_1,0
1,6560_2,0
2,6560_3,0
3,6560_4,0
4,6560_5,0


In [82]:
test_submission['IDAppended'] = test_submission['ID'].str.split('_').str[0]
test_submission['SepsisLabel'] = test_submission.groupby('IDAppended')['SepsisLabel'].transform(
    lambda x: (x.cumsum() > 0).astype(int)
)

In [83]:
test_submission = test_submission.drop('IDAppended', axis = 1)
test_submission['SepsisLabel'] = yHatTestScaled
test_submission.head()

,ID,SepsisLabel
0,6560_1,0
1,6560_2,0
2,6560_3,0
3,6560_4,0
4,6560_5,0


In [84]:
#!dir
test_submission.to_csv("../Data/rawData/ydf_submission.csv", index = False)

In [85]:
test_submission.shape

(464354, 2)